### Code Flow and Explanation:

1. **Import Libraries**:
   - `pandas` for data manipulation, `cosine_similarity` from `sklearn.metrics.pairwise` for similarity computation, and `numpy` for numerical operations.

2. **Load Data**:
   - Load the user-event interaction data from the Excel file `Combined_Interactions.xlsx`.

3. **Create User-Item Matrix**:
   - Pivot the data to form a **user-item interaction matrix** (`user_item_matrix`), where rows represent users, columns represent events, and values are interaction scores. Missing values are filled with 0.

4. **Compute User Similarity**:
   - Use **cosine similarity** to compute a similarity matrix between users based on their event interaction behavior.

5. **Recommend Events Function**:
   - The `recommend_events(user_index, top_n=10)` function recommends the top `n` events for a given user:
     - Find similar users by sorting based on similarity scores.
     - Check events that similar users have interacted with but the target user has not.
     - Accumulate recommendation scores based on the interactions of similar users.

6. **Recommendation Filtering**:
   - Exclude events the user has already interacted with or events with a score of `-1`.

7. **Sort Recommendations**:
   - Sort the event recommendations by interaction score in descending order and return the top `n` recommendations.

8. **Example Usage**:
   - Recommends events for the user `'Ketaki'` and prints the top recommendations.

In [5]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Load data
data = pd.read_excel('Combined_Interactions.xlsx')

# Filter out events with interaction score of -1
#data = data[data['Interaction Score'] != -1]

# Pivot table to create user-item interaction matrix
user_item_matrix = data.pivot_table(index='Name', columns='Event ID', values='Interaction Score', fill_value=0)

# Compute user-user similarity matrix using cosine similarity
user_similarity = cosine_similarity(user_item_matrix)

# Function to recommend events for a given user
# Function to recommend events for a given user
def recommend_events(user_index, top_n=10):
    # Find similar users
    similar_users = np.argsort(user_similarity[user_index])[::-1][1:]  # Exclude the user itself
    
    # Initialize dictionary to store event recommendations
    event_recommendations = {}
    
    # Get the events already interacted with by the user
    #interacted_events = user_item_matrix.columns[user_item_matrix.iloc[user_index].to_numpy().nonzero()]
    interacted_events = data[data['Name'] == user]['Event ID'].tolist()
    # Iterate over similar users and recommend events
    for similar_user in similar_users:
        for event_id in user_item_matrix.columns:
            if (user_item_matrix.iloc[user_index][event_id] == 0 and 
                user_item_matrix.iloc[similar_user][event_id] > 0 and 
                event_id not in interacted_events):  # Check if event is not already interacted with
                if event_id not in event_recommendations and user_item_matrix.iloc[similar_user][event_id] != -1:
                    event_recommendations[event_id] = user_item_matrix.iloc[similar_user][event_id]
                elif user_item_matrix.iloc[similar_user][event_id] != -1:
                    event_recommendations[event_id] += user_item_matrix.iloc[similar_user][event_id]
    
    # Sort recommendations by interaction score
    sorted_recommendations = sorted(event_recommendations.items(), key=lambda x: x[1], reverse=True)
    
    # Return top n recommendations
    return sorted_recommendations[:top_n]

# Example: Recommend events for user 'Vasudev'
user = 'Ketaki'
user_index = user_item_matrix.index.get_loc(user) # Get the index of the user in the user_item_matrix
recommended_events = recommend_events(user_index)
print("Recommended events for {}: {}".format(user, recommended_events))

Recommended events for Ketaki: [('EVT0022', 47), ('EVT0085', 45), ('EVT0030', 44), ('EVT0018', 41), ('EVT0020', 38), ('EVT0147', 34), ('EVT0025', 28), ('EVT0100', 25), ('EVT0142', 24), ('EVT0024', 20)]
